In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt                                                            
import time
import pyarrow.parquet as pq

## The HOURLY Dataset

In [2]:
parquet_file_path = 'C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\DatasetV01.parquet'

# Read the Parquet file into a PyArrow Table
table = pq.read_table(parquet_file_path)

# Convert the Table to a Pandas DataFrame if needed
df = table.to_pandas()

In [ ]:
df.columns

In [3]:
df_outputs=df[['Run', 'Date/Time',
       'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
       'Environment:Site Outdoor Air Relative Humidity [%](Hourly)',
       'Environment:Site Wind Speed [m/s](Hourly)',
       'Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly)',
       'Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly)',
       'Environment:Site Day Type Index [](Hourly)',       'Electricity:Facility [J](Hourly)', 'NaturalGas:Facility [J](Hourly)',
       'Heating:NaturalGas [J](Hourly)', 'Cooling:Electricity [J](Hourly)',
       'Heating:Electricity [J](Hourly)']]

In [ ]:
df_outputs.columns

In [4]:
df_outputs=df_outputs.rename(columns={'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'AirTemp (C)',
       'Environment:Site Outdoor Air Relative Humidity [%](Hourly)':'RH (%)',
       'Environment:Site Wind Speed [m/s](Hourly)':'WindSpeed (m/s)',
       'Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly)':'DiffuseRadiation(W/m2)',
       'Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly)':'DirectRadiation(W/m2)',
       'Environment:Site Day Type Index [](Hourly)':'DayType',
       'Electricity:Facility [J](Hourly)':'Electricity (J)',
       'NaturalGas:Facility [J](Hourly)':'NaturalGas (J)',
       'Heating:NaturalGas [J](Hourly)':'Heating:NaturalGas (J)', 
       'Cooling:Electricity [J](Hourly)':'Cooling:Electricity (J)',
       'Heating:Electricity [J](Hourly)':'Heating:Electricity (J)'})

In [ ]:
df_outputs['DayType'].unique()

In [5]:
## Saturday and Sunday schedules are different 
df_outputs['DayType'] =  np.where(df_outputs['DayType'].isin([ 2, 3, 4, 5, 6]), 1,           # If DayType is 2-6, set to 1
                         np.where(df_outputs['DayType'].isin([1, 8]), 0,                     # If DayType is 1 or 8, set to 0
                         np.where(df_outputs['DayType'] == 7, 0.5, df_outputs['DayType'])))  # If DayType is 7, set to 0.5

In [6]:
# Clean up the Date/Time column: remove leading/trailing spaces and replace multiple spaces with a single space
df_outputs['Date/Time'] = df_outputs['Date/Time'].str.strip().replace(r'\s+', ' ', regex=True)
# Handle '24:00:00' by replacing it with '00:00:00' and incrementing the date
df_outputs['Date/Time'] = df_outputs['Date/Time'].str.replace(' 24:00:00', ' 00:00:00')
df_outputs['Date/Time'] = pd.to_datetime(df_outputs['Date/Time'], format='%m/%d %H:%M:%S')
# Increment the day for timestamps that were originally '24:00:00'
mask = df_outputs['Date/Time'].dt.time == pd.to_datetime('00:00:00').time()
df_outputs.loc[mask, 'Date/Time'] += pd.Timedelta(days=1)

In [7]:
df_outputs

,Run,Date/Time,AirTemp (C),RH (%),WindSpeed (m/s),DiffuseRadiation(W/m2),DirectRadiation(W/m2),DayType,Electricity (J),NaturalGas (J),Heating:NaturalGas (J),Cooling:Electricity (J),Heating:Electricity (J)
0,0,1900-01-01 01:00:00,3.516667,95.916667,1.933333,0.0,0.0,0.0,1.474575e+08,7.200000e+04,0.000000,0.0,1.072356e+08
1,0,1900-01-01 02:00:00,1.616667,94.166667,1.450000,0.0,0.0,0.0,1.486428e+08,7.200000e+04,0.000000,0.0,1.083375e+08
2,0,1900-01-01 03:00:00,3.691667,96.166667,1.700000,0.0,0.0,0.0,1.732599e+08,4.070681e+06,0.000000,0.0,1.313769e+08
3,0,1900-01-01 04:00:00,4.866667,97.583333,1.700000,0.0,0.0,0.0,1.335554e+08,7.200000e+04,0.000000,0.0,9.412272e+07
4,0,1900-01-01 05:00:00,5.258333,98.583333,1.816667,0.0,0.0,0.0,1.678405e+08,4.070474e+06,0.000000,0.0,1.261757e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379995,499,1900-12-31 20:00:00,-8.925000,88.000000,5.350000,0.0,0.0,0.0,2.246467e+08,7.200000e+04,0.000000,0.0,1.836339e+08
4379996,499,1900-12-31 21:00:00,-8.566667,88.000000,4.900000,0.0,0.0,0.0,2.769176e+08,4.133570e+06,62975.228368,0.0,2.339322e+08
4379997,499,1900-12-31 22:00:00,-7.816667,88.583333,4.108333,0.0,0.0,0.0,2.178911e+08,7.200000e+04,0.000000,0.0,1.768784e+08
4379998,499,1900-12-31 23:00:00,-6.291667,89.583333,3.258333,0.0,0.0,0.0,2.593779e+08,4.070222e+06,0.000000,0.0,2.163925e+08


In [8]:
df_inputs= pd.read_csv("C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\InputsV01.csv")

In [9]:
## The index is the same as the run number which will be used for merging the datasets 
df_inputs.reset_index(inplace=True)

In [10]:
df_inputs.columns

Index(['index', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10'], dtype='object')

In [11]:
df_inputs=df_inputs.rename(columns={'index':'Run',
                                    'X1':'X1_WallR',
                                    'X2':'X2_RoofR',
                                    'X3':'X3_WindowU',
                                    'X4':'X4_SHGC',
                                    'X5':'X5_COPHigh',
                                    'X6':'X6_COPLow',
                                    'X7':'X7_HSP(7-21)',
                                    'X8':'X8_HSP(21-6)',
                                    'X9':'X9_CSP(6-21)',
                                    'X10':'X10_CSP(21-6)'})

In [12]:
df_inputs

,Run,X1_WallR,X2_RoofR,X3_WindowU,X4_SHGC,X5_COPHigh,X6_COPLow,X7_HSP(7-21),X8_HSP(21-6),X9_CSP(6-21),X10_CSP(21-6)
0,0,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
1,1,7.482,7.247,1.604,0.918,4.087,4.165,17.382,18.326,27.333,26.114
2,2,7.629,7.381,1.671,0.700,3.949,3.632,18.311,17.273,22.550,24.061
3,3,5.715,8.797,1.693,0.934,3.572,3.658,18.209,19.397,22.365,22.252
4,4,6.584,7.771,1.677,0.919,2.684,2.433,19.514,19.038,22.634,24.032
...,...,...,...,...,...,...,...,...,...,...,...
495,495,3.986,8.955,1.639,0.301,3.858,3.229,18.474,19.720,22.663,23.256
496,496,7.572,9.386,1.600,0.056,4.349,4.125,20.456,21.423,26.873,22.475
497,497,4.182,9.518,1.252,0.932,3.718,2.777,17.911,17.079,25.455,23.669
498,498,7.640,8.285,1.324,0.948,4.225,2.589,18.303,21.072,27.202,23.023


In [13]:
df_final=pd.merge(df_outputs,df_inputs, how='outer', on='Run')

In [14]:
df_final

,Run,Date/Time,AirTemp (C),RH (%),WindSpeed (m/s),DiffuseRadiation(W/m2),DirectRadiation(W/m2),DayType,Electricity (J),NaturalGas (J),...,X1_WallR,X2_RoofR,X3_WindowU,X4_SHGC,X5_COPHigh,X6_COPLow,X7_HSP(7-21),X8_HSP(21-6),X9_CSP(6-21),X10_CSP(21-6)
0,0,1900-01-01 01:00:00,3.516667,95.916667,1.933333,0.0,0.0,0.0,1.474575e+08,7.200000e+04,...,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
1,0,1900-01-01 02:00:00,1.616667,94.166667,1.450000,0.0,0.0,0.0,1.486428e+08,7.200000e+04,...,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
2,0,1900-01-01 03:00:00,3.691667,96.166667,1.700000,0.0,0.0,0.0,1.732599e+08,4.070681e+06,...,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
3,0,1900-01-01 04:00:00,4.866667,97.583333,1.700000,0.0,0.0,0.0,1.335554e+08,7.200000e+04,...,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
4,0,1900-01-01 05:00:00,5.258333,98.583333,1.816667,0.0,0.0,0.0,1.678405e+08,4.070474e+06,...,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379995,499,1900-12-31 20:00:00,-8.925000,88.000000,5.350000,0.0,0.0,0.0,2.246467e+08,7.200000e+04,...,4.017,7.253,1.478,0.453,2.800,3.813,18.574,17.738,22.202,26.474
4379996,499,1900-12-31 21:00:00,-8.566667,88.000000,4.900000,0.0,0.0,0.0,2.769176e+08,4.133570e+06,...,4.017,7.253,1.478,0.453,2.800,3.813,18.574,17.738,22.202,26.474
4379997,499,1900-12-31 22:00:00,-7.816667,88.583333,4.108333,0.0,0.0,0.0,2.178911e+08,7.200000e+04,...,4.017,7.253,1.478,0.453,2.800,3.813,18.574,17.738,22.202,26.474
4379998,499,1900-12-31 23:00:00,-6.291667,89.583333,3.258333,0.0,0.0,0.0,2.593779e+08,4.070222e+06,...,4.017,7.253,1.478,0.453,2.800,3.813,18.574,17.738,22.202,26.474


In [15]:
df_final.columns

Index(['Run', 'Date/Time', 'AirTemp (C)', 'RH (%)', 'WindSpeed (m/s)',
       'DiffuseRadiation(W/m2)', 'DirectRadiation(W/m2)', 'DayType',
       'Electricity (J)', 'NaturalGas (J)', 'Heating:NaturalGas (J)',
       'Cooling:Electricity (J)', 'Heating:Electricity (J)', 'X1_WallR',
       'X2_RoofR', 'X3_WindowU', 'X4_SHGC', 'X5_COPHigh', 'X6_COPLow',
       'X7_HSP(7-21)', 'X8_HSP(21-6)', 'X9_CSP(6-21)', 'X10_CSP(21-6)'],
      dtype='object')

In [15]:
df_final=df_final[['Run', 'Date/Time', 'AirTemp (C)', 'RH (%)', 'WindSpeed (m/s)',
                   'DiffuseRadiation(W/m2)', 'DirectRadiation(W/m2)', 'DayType', 'X1_WallR',
                    'X2_RoofR', 'X3_WindowU', 'X4_SHGC', 'X5_COPHigh', 'X6_COPLow',
                    'X7_HSP(7-21)', 'X8_HSP(21-6)', 'X9_CSP(6-21)', 'X10_CSP(21-6)', 'Electricity (J)', 'NaturalGas (J)', 'Heating:NaturalGas (J)',
                    'Cooling:Electricity (J)', 'Heating:Electricity (J)']]

In [16]:
df_final

,Run,Date/Time,AirTemp (C),RH (%),WindSpeed (m/s),DiffuseRadiation(W/m2),DirectRadiation(W/m2),DayType,X1_WallR,X2_RoofR,...,X6_COPLow,X7_HSP(7-21),X8_HSP(21-6),X9_CSP(6-21),X10_CSP(21-6),Electricity (J),NaturalGas (J),Heating:NaturalGas (J),Cooling:Electricity (J),Heating:Electricity (J)
0,0,1900-01-01 01:00:00,3.516667,95.916667,1.933333,0.0,0.0,0.0,5.948,9.867,...,3.435,19.168,19.793,23.400,22.025,1.474575e+08,7.200000e+04,0.000000,0.0,1.072356e+08
1,0,1900-01-01 02:00:00,1.616667,94.166667,1.450000,0.0,0.0,0.0,5.948,9.867,...,3.435,19.168,19.793,23.400,22.025,1.486428e+08,7.200000e+04,0.000000,0.0,1.083375e+08
2,0,1900-01-01 03:00:00,3.691667,96.166667,1.700000,0.0,0.0,0.0,5.948,9.867,...,3.435,19.168,19.793,23.400,22.025,1.732599e+08,4.070681e+06,0.000000,0.0,1.313769e+08
3,0,1900-01-01 04:00:00,4.866667,97.583333,1.700000,0.0,0.0,0.0,5.948,9.867,...,3.435,19.168,19.793,23.400,22.025,1.335554e+08,7.200000e+04,0.000000,0.0,9.412272e+07
4,0,1900-01-01 05:00:00,5.258333,98.583333,1.816667,0.0,0.0,0.0,5.948,9.867,...,3.435,19.168,19.793,23.400,22.025,1.678405e+08,4.070474e+06,0.000000,0.0,1.261757e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379995,499,1900-12-31 20:00:00,-8.925000,88.000000,5.350000,0.0,0.0,0.0,4.017,7.253,...,3.813,18.574,17.738,22.202,26.474,2.246467e+08,7.200000e+04,0.000000,0.0,1.836339e+08
4379996,499,1900-12-31 21:00:00,-8.566667,88.000000,4.900000,0.0,0.0,0.0,4.017,7.253,...,3.813,18.574,17.738,22.202,26.474,2.769176e+08,4.133570e+06,62975.228368,0.0,2.339322e+08
4379997,499,1900-12-31 22:00:00,-7.816667,88.583333,4.108333,0.0,0.0,0.0,4.017,7.253,...,3.813,18.574,17.738,22.202,26.474,2.178911e+08,7.200000e+04,0.000000,0.0,1.768784e+08
4379998,499,1900-12-31 23:00:00,-6.291667,89.583333,3.258333,0.0,0.0,0.0,4.017,7.253,...,3.813,18.574,17.738,22.202,26.474,2.593779e+08,4.070222e+06,0.000000,0.0,2.163925e+08


In [17]:
df_final.to_csv("C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\HourlyV01.csv", index=False)

## The ANNUAL Dataset

In [2]:
df_out=pd.read_csv('C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\Annual_V01.csv')

In [3]:
df_inputs= pd.read_csv("C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\InputsV01.csv")

In [4]:
## The index is the same as the run number which will be used for merging the datasets 
df_inputs.reset_index(inplace=True)

In [5]:
df_inputs=df_inputs.rename(columns={'index':'Run',
                                    'X1':'X1_WallR',
                                    'X2':'X2_RoofR',
                                    'X3':'X3_WindowU',
                                    'X4':'X4_SHGC',
                                    'X5':'X5_COPHigh',
                                    'X6':'X6_COPLow',
                                    'X7':'X7_HSP(7-21)',
                                    'X8':'X8_HSP(21-6)',
                                    'X9':'X9_CSP(6-21)',
                                    'X10':'X10_CSP(21-6)'})

In [6]:
df_out

,Run,Total_Site_Energy (GJ),Total_Natural_Gas (GJ),Total_Electricity (GJ),Unmet_ASHRAE_55 (hr),Peak_Electricity (W)
0,0,1944.07,251.32,1692.75,2452.83,184351.94
1,1,1718.66,132.73,1585.93,3196.83,224329.50
2,2,1786.40,169.86,1616.54,2442.67,241195.76
3,3,1948.75,115.53,1833.22,2556.83,203262.92
4,4,2027.96,107.33,1920.63,2264.67,227034.83
...,...,...,...,...,...,...
495,495,1971.11,277.39,1693.72,2453.33,184141.25
496,496,2071.55,360.18,1711.36,2209.50,193512.03
497,497,1747.03,124.15,1622.88,3142.33,239067.68
498,498,2032.54,103.15,1929.40,3066.67,243260.81


In [7]:
data=pd.merge(df_inputs, df_out, how='outer', on='Run')

In [8]:
data.columns

Index(['Run', 'X1_WallR', 'X2_RoofR', 'X3_WindowU', 'X4_SHGC', 'X5_COPHigh',
       'X6_COPLow', 'X7_HSP(7-21)', 'X8_HSP(21-6)', 'X9_CSP(6-21)',
       'X10_CSP(21-6)', 'Total_Site_Energy (GJ)', 'Total_Natural_Gas (GJ)',
       'Total_Electricity (GJ)', 'Unmet_ASHRAE_55 (hr)',
       'Peak_Electricity (W)'],
      dtype='object')

In [9]:
data=data[['Run', 'X1_WallR', 'X2_RoofR', 'X3_WindowU', 'X4_SHGC', 'X5_COPHigh',
       'X6_COPLow', 'X7_HSP(7-21)', 'X8_HSP(21-6)', 'X9_CSP(6-21)',
       'X10_CSP(21-6)','Total_Natural_Gas (GJ)','Unmet_ASHRAE_55 (hr)','Peak_Electricity (W)']]

In [10]:
data

,Run,X1_WallR,X2_RoofR,X3_WindowU,X4_SHGC,X5_COPHigh,X6_COPLow,X7_HSP(7-21),X8_HSP(21-6),X9_CSP(6-21),X10_CSP(21-6),Total_Natural_Gas (GJ),Unmet_ASHRAE_55 (hr),Peak_Electricity (W)
0,0,5.948,9.867,1.652,0.357,4.446,3.435,19.168,19.793,23.400,22.025,251.32,2452.83,184351.94
1,1,7.482,7.247,1.604,0.918,4.087,4.165,17.382,18.326,27.333,26.114,132.73,3196.83,224329.50
2,2,7.629,7.381,1.671,0.700,3.949,3.632,18.311,17.273,22.550,24.061,169.86,2442.67,241195.76
3,3,5.715,8.797,1.693,0.934,3.572,3.658,18.209,19.397,22.365,22.252,115.53,2556.83,203262.92
4,4,6.584,7.771,1.677,0.919,2.684,2.433,19.514,19.038,22.634,24.032,107.33,2264.67,227034.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,3.986,8.955,1.639,0.301,3.858,3.229,18.474,19.720,22.663,23.256,277.39,2453.33,184141.25
496,496,7.572,9.386,1.600,0.056,4.349,4.125,20.456,21.423,26.873,22.475,360.18,2209.50,193512.03
497,497,4.182,9.518,1.252,0.932,3.718,2.777,17.911,17.079,25.455,23.669,124.15,3142.33,239067.68
498,498,7.640,8.285,1.324,0.948,4.225,2.589,18.303,21.072,27.202,23.023,103.15,3066.67,243260.81


In [11]:
data.to_csv("C:\\Users\\Elin Markarian\\OneDrive - Carleton University\\Optimization_Seif\\Datasets\\Dataset_01\\AnnualV01_Final.csv", index=False)